In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.utils import shuffle

In [139]:
dataTrain=pd.read_csv("data/Train.txt",sep="#|;",engine="python",header=None,index_col = False)
dataTest=pd.read_csv("data/Test.txt",sep="#|;",engine="python",header=None,index_col = False)
dataCros=pd.read_csv("data/Cross_Validation.txt",sep="#|;",engine="python",header=None,index_col = False)

data=shuffle(pd.concat([dataTrain,dataTest,dataCros]))
data.reset_index(inplace=True)

In [141]:
data.drop("index",axis=1,inplace=True)

In [142]:
data

,0,1,2,3,4,5,6,7,8,9,...,537,538,539,540,541,542,543,544,545,546
0,15,1.174268,3.615161,7.469300,-1.002361,-5.435337,-5.559184,-2.223504,0.979845,-0.479108,...,-4.784175,1.285400,-0.419497,-0.704285,-1.181814,-1.996084,0.799858,-0.800359,-0.684468,3.871443
1,18,20.779759,-2.352498,5.856809,1.586740,-9.151673,-5.087730,0.387474,-0.311995,-1.821276,...,-3.557101,-1.301274,-1.987934,-1.434603,-1.061169,-0.327106,1.820435,-0.876320,-0.032822,4.587909
2,2,-15.145963,-5.309125,0.704886,-4.849989,2.405115,-0.472650,-3.829461,0.993042,-1.543625,...,0.778177,-0.025693,-0.566019,-1.095286,1.498866,0.735872,-1.742573,0.842601,2.840785,3.380489
3,21,5.370569,-5.555590,-5.464175,-5.640558,-5.779774,-4.697760,-2.375463,-0.316436,-4.285427,...,-1.639081,-3.681968,-3.638710,5.718969,-1.831309,1.706116,0.086407,0.212828,0.636832,3.394390
4,17,12.069178,5.497075,13.435719,-4.237238,-7.670982,0.579941,-2.696431,2.515508,0.039407,...,-7.410639,-0.043979,5.692146,-4.671843,0.584809,-1.624623,0.346433,0.191979,0.981208,3.255674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20184,13,8.987195,1.184439,-0.677812,-12.229966,-2.268233,0.217330,1.920094,2.186353,-1.515964,...,-3.601483,0.827391,1.424124,-2.305297,-1.234242,3.686427,0.116671,-1.305248,0.888827,3.537471
20185,10,-8.508996,-0.932560,5.076442,-3.897549,-2.545925,0.086706,-0.306069,1.816755,1.397750,...,-1.192028,1.122005,-2.235725,3.036916,5.759916,1.270373,-4.796707,1.169153,-0.594243,3.547046
20186,14,-10.414461,-4.714322,-3.460985,1.670220,0.066598,-3.170931,3.845575,1.259415,0.848282,...,0.306260,-1.552964,0.898969,2.216609,-0.767540,2.066600,0.125317,0.393434,-0.199751,3.727657
20187,10,-4.788495,-6.786077,-1.136508,-3.936910,2.958965,-3.521690,1.248235,1.345330,0.686123,...,-8.891863,0.428952,-3.739255,-1.339330,1.821382,0.974285,2.122856,0.443067,1.671430,4.766633


In [4]:
def find_best_model(learning_algorithm,X,Y,expected_accuracy=0.75,model_selection=None,model_selection_params={},params={}):
    import pickle
    while True:
        train_size=np.round(np.random.random(500),2)
        train_size = train_size[(train_size>0.5)&(train_size<0.95)]
        for size in train_size:
            X_train,X_val,y_train,y_val = train_test_split(X,Y,train_size=size)
            
            #adding default parametrs If params is not empty and model_selection is not None
            if model_selection:
                for (key,value) in params.items():
                    model = learning_algorithm()
                    l=set()
                    if hasattr(value,"__iter__") and type(value)!=str:
                        l=set(value)
                    else:
                        l.add(value)
                    l.add(model.__dict__.get(key))
                    params[key]=list(l)
                selected_model = model_selection(model,params,scoring="accuracy",**model_selection_params)
                model = selected_model.fit(X_train,y_train)
            else:
                model = learning_algorithm(**params)
                model.fit(X_train,y_train)
            
            y_val_predict=model.predict(X_val)
            val_accuracy=accuracy_score(y_val_predict,y_val)

            print(f"Train Size: {size}, Model: {learning_algorithm.__name__}, Test Accuracy:{val_accuracy:.3}")
            if val_accuracy>=expected_accuracy:
                return val_accuracy,model

In [171]:
X=data.drop(0,axis=1).copy()
Y=data[0].copy()
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,train_size=0.8)

In [47]:
Y_bin_train=Y_train.copy()
Y_bin_train[Y_bin_train<11]=0
Y_bin_train[Y_bin_train>=11]=1

Y_bin_test=Y_test.copy()
Y_bin_test[Y_bin_test<11]=0
Y_bin_test[Y_bin_test>=11]=1

In [10]:
accuracy,model_chooser=find_best_model(XGBClassifier,X_train,Y_bin_train,expected_accuracy=0.984)

Train Size: 0.64, Model: XGBClassifier, Test Accuracy:0.976
Train Size: 0.82, Model: XGBClassifier, Test Accuracy:0.982
Train Size: 0.84, Model: XGBClassifier, Test Accuracy:0.979
Train Size: 0.57, Model: XGBClassifier, Test Accuracy:0.976
Train Size: 0.86, Model: XGBClassifier, Test Accuracy:0.981
Train Size: 0.51, Model: XGBClassifier, Test Accuracy:0.977
Train Size: 0.51, Model: XGBClassifier, Test Accuracy:0.972
Train Size: 0.74, Model: XGBClassifier, Test Accuracy:0.98
Train Size: 0.51, Model: XGBClassifier, Test Accuracy:0.974
Train Size: 0.53, Model: XGBClassifier, Test Accuracy:0.975
Train Size: 0.52, Model: XGBClassifier, Test Accuracy:0.973
Train Size: 0.76, Model: XGBClassifier, Test Accuracy:0.979
Train Size: 0.74, Model: XGBClassifier, Test Accuracy:0.981
Train Size: 0.73, Model: XGBClassifier, Test Accuracy:0.979
Train Size: 0.79, Model: XGBClassifier, Test Accuracy:0.981
Train Size: 0.67, Model: XGBClassifier, Test Accuracy:0.979
Train Size: 0.67, Model: XGBClassifier, T

In [11]:
y_predict=model_chooser.predict(X_test)
accuracy_score(y_predict,Y_bin_test)

0.9826646854878652

In [143]:
train_data=X_train
train_data[0]=Y_train

test_data=X_test
test_data[0]=Y_test

In [145]:
test_data

,1,2,3,4,5,6,7,8,9,10,...,538,539,540,541,542,543,544,545,546,0
2272,-18.819280,-3.250937,-2.996723,-3.755008,-1.950222,1.757228,1.321905,1.402989,0.697494,0.627933,...,1.343116,-1.837249,-1.536591,-1.147146,-0.632510,-5.126932,0.392374,2.445716,4.945158,5
12162,-2.733385,-12.511544,8.439544,3.301367,-6.479287,-1.412634,-2.254308,1.241768,-1.664820,1.042096,...,-4.519411,2.887867,-5.217552,0.021271,-0.343936,1.494027,0.630348,0.667979,2.999143,11
2248,-27.570297,1.761105,0.099217,-0.410629,3.448492,-4.215263,1.461226,-3.329019,1.291122,-1.678540,...,4.686956,0.151791,0.383688,-2.141752,-1.186961,-1.446684,1.472838,-0.991017,4.729531,7
4716,15.018356,2.464844,7.907744,-1.021957,-2.587444,-7.955678,-3.987988,-2.542790,0.492707,1.353814,...,5.601329,0.141085,2.790363,0.820347,1.126670,-1.180898,2.512608,1.359650,5.015584,19
2833,3.194816,-5.035520,8.112346,1.029398,-2.408616,0.165253,2.233233,1.096309,1.099075,-2.272716,...,0.176503,1.359030,-0.258948,-1.851477,1.007868,-0.470776,2.109336,1.850506,5.820916,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11974,5.329143,4.518270,6.621529,-0.141847,0.291944,-4.104183,-0.631401,0.152410,0.866037,1.622545,...,1.981160,-0.163145,1.561310,2.096590,0.597258,-0.289310,0.913203,0.479520,3.864861,19
19998,-2.917509,-1.527286,4.726851,-3.298331,-3.471540,-6.423666,-1.625799,3.230384,-0.164592,-0.571496,...,3.456582,-0.915088,-0.559946,-1.119867,1.020337,1.182584,2.364554,0.813728,4.522223,15
12991,3.415525,-2.935208,-5.173124,-7.449270,-1.224885,2.857586,-1.661464,0.633913,-2.166953,-1.004927,...,-1.534284,1.615508,-2.886603,-0.475759,1.782169,-1.814236,3.178264,-0.391625,3.847485,12
8466,-4.499894,7.834021,8.659677,1.025180,3.279251,-4.792394,-4.788613,1.764706,0.238235,-1.735270,...,2.056694,0.542295,0.318407,-1.014801,-0.243846,-0.363568,-0.402315,1.679294,3.446845,0


In [151]:
Y_xgb_train=train_data[train_data[0]<11][0]
X_xgb_train=train_data[train_data[0]<11].drop(0,axis=1)

In [152]:
X_xgb_train

,1,2,3,4,5,6,7,8,9,10,...,537,538,539,540,541,542,543,544,545,546
7044,-10.317473,-2.282328,1.163257,-0.418499,0.321938,-2.264991,0.886754,0.582861,-0.191923,1.406373,...,-0.091983,-2.320935,-1.095586,-0.624890,1.772184,0.621533,0.021707,0.209873,-1.771687,4.614632
2930,-24.254484,2.013283,-3.480892,0.285683,3.849994,-5.321322,2.423151,-2.441727,-2.046678,-0.359238,...,2.751408,0.813872,0.601109,1.566180,5.540698,1.538407,0.515837,0.835611,0.317316,4.362038
16005,2.778416,8.919006,4.629494,-0.894167,-2.142428,-2.802478,-1.743985,-1.297653,-0.202003,-0.322324,...,3.099313,3.763424,3.549419,0.315160,0.263111,1.539059,0.091824,-1.313167,0.457084,5.052913
16260,-0.202360,1.117495,-1.532384,3.520753,2.955473,1.459185,2.137268,0.416097,1.254988,0.646371,...,-1.215055,2.717771,2.097070,0.018507,-1.446014,0.535163,0.623871,0.748296,-0.180348,4.574568
5583,0.383956,-0.630305,11.142123,1.024071,4.147324,2.056039,0.328327,3.033041,2.913605,-2.255384,...,-4.207780,3.704534,2.275140,2.664640,1.559429,3.192041,2.728832,1.863523,0.153430,3.376920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3977,-25.823525,-3.860514,-2.388569,-4.109286,1.302028,-9.438806,0.286343,-1.098603,1.864414,1.891722,...,4.883569,3.853566,0.826096,-0.844008,1.447167,3.589905,3.401262,2.082740,1.097911,4.177935
17441,3.656995,-6.172488,3.456050,-1.923188,-1.919121,0.370924,-0.913673,-2.026361,-1.270042,0.898985,...,11.199341,2.481503,-5.217887,-0.827030,3.767723,0.399302,0.646617,0.004995,-1.406234,3.525545
11571,-26.239006,7.612934,-2.143486,0.556835,6.303131,-0.291018,1.112148,-0.860067,-1.075120,-2.475230,...,4.509127,2.627880,-2.489301,0.311665,-0.896024,-1.053053,0.630197,1.890053,-2.198515,3.232887
16523,-4.839686,-3.293221,1.259990,1.590286,0.969730,0.904081,-0.948582,-1.534156,-1.044786,-0.740326,...,7.410908,5.435093,-0.692111,1.364499,-0.134020,0.910467,-1.911963,1.067671,2.701431,4.261802


In [153]:
Y_xgb_test=test_data[test_data[0]<11][0]
X_xgb_test=test_data[test_data[0]<11].drop(0,axis=1)

In [154]:
accuracy,model1=find_best_model(XGBClassifier,X_xgb_train,Y_xgb_train,expected_accuracy=0.975)

Train Size: 0.86, Model: XGBClassifier, Test Accuracy:0.972
Train Size: 0.8, Model: XGBClassifier, Test Accuracy:0.973
Train Size: 0.67, Model: XGBClassifier, Test Accuracy:0.969
Train Size: 0.78, Model: XGBClassifier, Test Accuracy:0.969
Train Size: 0.64, Model: XGBClassifier, Test Accuracy:0.967
Train Size: 0.81, Model: XGBClassifier, Test Accuracy:0.975


In [155]:
accuracy_score(model1.predict(X_xgb_test),Y_xgb_test)

0.9699326075686885

In [156]:
X_xgb1_train=train_data[train_data[0]>=11].drop(0,axis=1)
Y_xgb1_train=train_data[train_data[0]>=11][0]

In [157]:
#Invalid classes inferred from unique values of `y`.  
#Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13], got [11 12 13 14 15 16 17 18 19 20 21 22 23]
#Y_xgb1_train dəyərləri [11-23] aralığında olduğu üçün [0-12] aralığına gətirmək üçün 11 çıxılır.
#accuracy_score üçünsə isə modelin tapdığı dəyərlərin üzərinə 11 əlavə edilir
accuracy,model2=find_best_model(XGBClassifier,X_xgb1_train,Y_xgb1_train-11,expected_accuracy=0.975)

Train Size: 0.84, Model: XGBClassifier, Test Accuracy:0.967
Train Size: 0.77, Model: XGBClassifier, Test Accuracy:0.973
Train Size: 0.71, Model: XGBClassifier, Test Accuracy:0.972
Train Size: 0.53, Model: XGBClassifier, Test Accuracy:0.964
Train Size: 0.87, Model: XGBClassifier, Test Accuracy:0.974
Train Size: 0.79, Model: XGBClassifier, Test Accuracy:0.976


In [108]:
X_2_test=test_data[test_data[546]>=11].drop(546,axis=1)
Y_2_test=test_data[test_data[546]>=11][546]
predict_=model2.predict(X_2_test)+11
accuracy_score(predict_,Y_2_test)

0.9767116907312529

In [191]:
class Model():
    def __init__(self,model_chooser,model1,model2):
        self.model_chooser=model_chooser
        self.model1=model1
        self.model2=model2
    def predict(self,X):
        data=pd.DataFrame(X.copy())
        data[0]=np.nan
        data[0]=self.model_chooser.predict(data.drop(0,axis=1))
        X_zeros=data[data[0]==0].drop(0,axis=1)
        X_ones=data[data[0]==1].drop(0,axis=1)

        data.loc[data[0]==1,0]=-1
        if not X_zeros.empty:
            data.loc[data[0]==0,0] = self.model1.predict(X_zeros)
    
        if not X_ones.empty:
            data.loc[data[0]==-1,0] = self.model2.predict(X_ones)+11
            
        return data[0]

In [193]:
model=Model(model_chooser,model1,model2)
y_predict=model.predict(X_test)

In [195]:
import pickle

In [198]:
with open("xgbclassifier.pkl","rb") as file:
    module_=pickle.load(file)

In [199]:
y_predict = module_.predict(X_test)

In [192]:
X_test

,1,2,3,4,5,6,7,8,9,10,...,537,538,539,540,541,542,543,544,545,546
2926,-10.324040,-1.684870,-1.051042,0.191942,-1.472049,0.339222,-1.516661,1.512051,-1.147149,-0.718827,...,-4.429662,-0.671928,0.458984,3.016104,0.526826,-1.653703,-1.894530,-0.463444,-1.270911,6.406639
4442,9.356213,-10.131303,-3.560539,-9.074752,-1.481468,5.986705,-0.779703,-0.707774,-1.898338,-1.176248,...,-3.463708,-2.446212,-4.091929,-8.043791,2.701569,-5.428819,0.201552,-2.929698,-0.515843,1.596285
19653,1.976460,1.057189,5.544438,3.253305,2.929040,-0.281639,2.014333,1.229921,0.374294,2.295471,...,-0.677064,-4.066100,-1.168931,2.334999,-1.976627,-3.165512,1.419012,0.524960,-0.277933,4.611310
11262,-20.805114,-2.051127,-6.063682,1.353782,-5.088858,1.370855,2.998812,1.181821,1.784570,0.494495,...,8.733566,2.003448,-1.012143,-0.596596,0.363548,-0.913001,0.254272,1.914432,-0.507613,3.087279
12253,-16.844257,2.405521,-1.664123,0.327079,-1.769487,0.781457,-0.040832,2.334993,0.111600,0.646212,...,-1.066786,-3.762727,-0.099544,0.182458,0.638387,-0.867175,-1.839330,-0.707961,0.386442,5.365601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7948,-24.837914,4.732775,-2.160193,-1.793081,-2.218461,-1.029702,1.486864,0.103762,-2.035089,-1.692551,...,0.150989,-3.376907,-3.485806,-4.311451,-3.272468,-6.771069,-6.095440,-4.211384,-3.852276,2.371386
6943,-2.287607,-16.162886,4.058274,2.481238,-6.033628,-2.404577,0.920536,-1.338708,-3.060704,-0.025069,...,-0.332731,2.709209,0.068081,1.643229,0.077615,0.513487,0.263376,2.038176,0.571274,4.289493
14697,-0.466428,-2.606912,-7.382547,-2.624519,0.781286,-0.991549,-2.197799,-0.497306,-1.090514,0.509039,...,-7.160431,-0.058666,-2.885780,-4.157996,0.582780,-0.406214,0.199491,1.325208,1.477460,2.446261
12459,-13.014002,-10.219773,-4.021554,0.563281,-1.253751,-0.397729,-0.767202,-0.079599,0.437100,1.048372,...,-1.048219,1.634408,2.209734,1.508889,0.895299,0.631725,-1.104731,-2.667836,1.748405,5.006951


In [200]:
accuracy_score(y_predict,Y_test)

0.9876176324913324

In [183]:
X

,1,2,3,4,5,6,7,8,9,10,...,537,538,539,540,541,542,543,544,545,546
0,1.174268,3.615161,7.469300,-1.002361,-5.435337,-5.559184,-2.223504,0.979845,-0.479108,0.066500,...,-4.784175,1.285400,-0.419497,-0.704285,-1.181814,-1.996084,0.799858,-0.800359,-0.684468,3.871443
1,20.779759,-2.352498,5.856809,1.586740,-9.151673,-5.087730,0.387474,-0.311995,-1.821276,2.099557,...,-3.557101,-1.301274,-1.987934,-1.434603,-1.061169,-0.327106,1.820435,-0.876320,-0.032822,4.587909
2,-15.145963,-5.309125,0.704886,-4.849989,2.405115,-0.472650,-3.829461,0.993042,-1.543625,-1.405689,...,0.778177,-0.025693,-0.566019,-1.095286,1.498866,0.735872,-1.742573,0.842601,2.840785,3.380489
3,5.370569,-5.555590,-5.464175,-5.640558,-5.779774,-4.697760,-2.375463,-0.316436,-4.285427,-2.954198,...,-1.639081,-3.681968,-3.638710,5.718969,-1.831309,1.706116,0.086407,0.212828,0.636832,3.394390
4,12.069178,5.497075,13.435719,-4.237238,-7.670982,0.579941,-2.696431,2.515508,0.039407,-2.736248,...,-7.410639,-0.043979,5.692146,-4.671843,0.584809,-1.624623,0.346433,0.191979,0.981208,3.255674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20184,8.987195,1.184439,-0.677812,-12.229966,-2.268233,0.217330,1.920094,2.186353,-1.515964,-2.199157,...,-3.601483,0.827391,1.424124,-2.305297,-1.234242,3.686427,0.116671,-1.305248,0.888827,3.537471
20185,-8.508996,-0.932560,5.076442,-3.897549,-2.545925,0.086706,-0.306069,1.816755,1.397750,-0.131586,...,-1.192028,1.122005,-2.235725,3.036916,5.759916,1.270373,-4.796707,1.169153,-0.594243,3.547046
20186,-10.414461,-4.714322,-3.460985,1.670220,0.066598,-3.170931,3.845575,1.259415,0.848282,-0.127345,...,0.306260,-1.552964,0.898969,2.216609,-0.767540,2.066600,0.125317,0.393434,-0.199751,3.727657
20187,-4.788495,-6.786077,-1.136508,-3.936910,2.958965,-3.521690,1.248235,1.345330,0.686123,-2.693097,...,-8.891863,0.428952,-3.739255,-1.339330,1.821382,0.974285,2.122856,0.443067,1.671430,4.766633


In [133]:
X

,1,2,3,4,5,6,7,8,9,10,...,537,538,539,540,541,542,543,544,545,546
0,-21.025677,-7.773474,7.133387,-4.671318,-0.374026,-1.090159,0.183514,-0.059586,-1.446150,0.371630,...,-0.323301,1.702649,-2.787289,-3.413489,0.801250,1.758936,1.322757,-1.566186,0.268926,NaN
1,-6.239406,-10.470883,-1.580303,-1.692950,-1.133805,-0.541795,0.283067,-1.121666,-0.596858,-0.961028,...,-3.163461,-0.056743,-3.347577,-0.932688,-1.035909,1.016222,1.086680,-0.815874,1.809152,NaN
2,-4.552481,0.776348,3.690149,-1.111741,-2.041460,-1.945581,-2.484114,1.648486,1.208859,-0.214560,...,0.070395,0.587563,1.964908,2.019767,1.077837,0.836551,-0.492015,0.762119,1.150268,NaN
3,10.787390,1.237523,5.009998,-1.120608,-6.524876,-5.216195,-0.708635,-0.995203,1.708172,0.729100,...,-4.412763,-4.949654,-3.945429,-1.043257,0.608999,2.028053,-0.061999,-2.618906,-2.692145,NaN
4,5.364697,-11.760070,4.549422,-5.680356,-4.422503,2.293858,-4.530885,3.868488,2.148298,-1.860124,...,-1.741108,-3.855110,-3.852412,-1.725982,-0.631623,0.487776,-0.669433,0.186811,1.440233,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20184,-9.766406,-9.971387,4.518501,-0.091920,-4.558230,-2.175913,0.477728,-0.821876,1.543057,0.694396,...,-0.943271,1.479182,3.709796,3.373929,-0.844310,-2.480219,-1.539992,-1.004613,-0.075404,NaN
20185,2.007948,-4.069488,-8.822362,0.132023,6.791959,-2.175457,-0.846486,-1.007688,2.719432,1.380734,...,-0.985319,-3.259454,-4.273387,0.657249,-0.326951,1.421581,-3.357706,1.333841,1.107678,NaN
20186,7.995357,-1.241242,-3.781893,-11.367960,-0.462412,5.431892,-1.028806,-0.697524,-0.022817,-0.360604,...,3.914156,3.895152,3.688111,0.816443,0.393289,-0.115019,0.404894,-1.075229,0.191504,NaN
20187,6.722099,7.099822,-2.672320,-10.762996,-4.002316,-3.270559,4.419567,-3.134800,-2.641513,1.532505,...,2.688534,4.013864,-0.653121,0.051428,0.111086,1.533178,0.427940,0.161484,1.044348,NaN
